<a href="https://colab.research.google.com/github/Esantomi/digital-humanities/blob/main/%EC%9D%B8%EB%AC%B8%EC%A7%80%EC%8B%9D%EC%B2%98%EB%A6%AC%EC%99%80%ED%86%B5%EA%B3%84/%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D/%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94_OKT_%EC%82%AC%EC%9A%A9%EC%9E%90%EC%82%AC%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비(사용자사전)

https://konlpy.org/ko/



In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

     |████████████████████████████████| 2.8MB 6.4MB/s 
     |████████████████████████████████| 19.4MB 23.9MB/s 
     |████████████████████████████████| 92kB 6.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/open?id=1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK

# https://drive.google.com/file/d/1U4EJIBj3gu9OVNjO8ubEbrnlKAbssdHO/view?usp=sharing

rawdata_downloaded = drive.CreateFile({'id': '1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK'})
rawdata_downloaded.GetContentFile('rawdata.tsv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.tsv', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,id,type,text
0,1,노무현,존경하는 국민 여러분. 오늘 저는 대한민국의 제16대 대통령에 취임하기 위해 이...
1,2,이명박,"존경하는 국민 여러분! 700만 해외동포 여러분, 이 자리에 참석하신 노무현, 김..."
2,3,박근혜,희망의 새 시대를 열겠습니다. 존경하는 국민 여러분! 700만 해외동포 여러분! ...
3,4,문재인,"존경하고 사랑하는 국민 여러분, 감사합니다. 국민 여러분의 위대한 선택에 머리 숙여..."


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # 존경하는 국민 여러분. 오늘 저는 대한민국의 제16대 대통령에 취임하기 위해 이...
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    #노무현
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,존경,Noun,문재인,1
1,하고,Josa,문재인,1
2,사랑,Noun,문재인,1
3,하는,Verb,문재인,1
4,국민,Noun,문재인,1
...,...,...,...,...
1198,일,Noun,문재인,1
1199,하겠습니다,Verb,문재인,1
1200,.,Punctuation,문재인,1
1201,감사합니다,Verb,문재인,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[          0            1 type  count
 0        존경         Noun  노무현      1
 1        하는         Verb  노무현      1
 2        국민         Noun  노무현      1
 3       여러분         Noun  노무현      1
 4         .  Punctuation  노무현      1
 ...     ...          ...  ...    ...
 2223     맹세         Noun  노무현      1
 2224    합니다         Verb  노무현      1
 2225      .  Punctuation  노무현      1
 2226  감사합니다         Verb  노무현      1
 2227      .  Punctuation  노무현      1
 
 [2228 rows x 4 columns],           0            1 type  count
 0        존경         Noun  이명박      1
 1        하는         Verb  이명박      1
 2        국민         Noun  이명박      1
 3       여러분         Noun  이명박      1
 4         !  Punctuation  이명박      1
 ...     ...          ...  ...    ...
 3586      것         Noun  이명박      1
 3587    입니다    Adjective  이명박      1
 3588      .  Punctuation  이명박      1
 3589  감사합니다         Verb  이명박      1
 3590      .  Punctuation  이명박      1
 
 [3591 rows x 4 columns],           0            1 type  co

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,존경,Noun,노무현,1
1,하는,Verb,노무현,1
2,국민,Noun,노무현,1
3,여러분,Noun,노무현,1
4,.,Punctuation,노무현,1
...,...,...,...,...
1198,일,Noun,문재인,1
1199,하겠습니다,Verb,문재인,1
1200,.,Punctuation,문재인,1
1201,감사합니다,Verb,문재인,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,존경,Noun,노무현,1
1,하는,Verb,노무현,1
2,국민,Noun,노무현,1
3,여러분,Noun,노무현,1
4,.,Punctuation,노무현,1
...,...,...,...,...
1198,일,Noun,문재인,1
1199,하겠습니다,Verb,문재인,1
1200,.,Punctuation,문재인,1
1201,감사합니다,Verb,문재인,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소     품사           분류 
!       Punctuation  박근혜     9
                     이명박     9
'       Punctuation  노무현    10
(       Punctuation  이명박     4
)       Punctuation  이명박     4
                            ..
힘       Noun         이명박     5
힘들었던    Adjective    문재인     1
힘쓰겠습니다  Verb         이명박     2
힘쓸      Verb         이명박     1
힘차게     Adjective    이명박     1
Name: 카운트, Length: 3285, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소    품사          분류      
!      Punctuation 박근혜    9
                   이명박    9
'      Punctuation 노무현   10
(      Punctuation 이명박    4
)      Punctuation 이명박    4
...                     ...
힘      Noun        이명박    5
힘들었던   Adjective   문재인    1
힘쓰겠습니다 Verb        이명박    2
힘쓸     Verb        이명박    1
힘차게    Adjective   이명박    1

[3285 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')